# Automatización de Reportes y Dashboard para Modelos de Biomasa
## Flujo completo: Datos → Modelado → Reportes Excel → Dashboard

In [ ]:
# Librerías para análisis y modelado
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Librerías para automatización de reportes
import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.chart import BarChart, LineChart, Reference, ScatterChart
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.drawing.image import Image
import io
import base64

# Librerías ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, classification_report
from sklearn.impute import SimpleImputer

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Librerías cargadas correctamente")

## 1. Carga y Preprocesamiento de Datos

In [ ]:
# Función para cargar y preparar datos
def cargar_y_preparar_datos(archivo):
    """Carga y prepara los datos para modelado"""
    try:
        df = pd.read_excel(archivo, sheet_name='Datos Limpios')
        print(f"✅ Datos cargados: {df.shape}")
        
        # Preparar datos para regresión
        df_reg = df.copy()
        features_to_drop = ['Fecha de Medicion', 'ID_parcela', 'Categoria de Biomasa', 'Validacion']
        df_reg = df_reg.drop(columns=[col for col in features_to_drop if col in df_reg.columns])
        
        target_reg = 'Biomasa_real Estadistica'
        y_reg = df_reg[target_reg]
        X_reg = df_reg.drop(columns=[target_reg])
        
        # Preparar datos para clasificación
        df_clf = df.copy()
        features_to_drop_clf = ['Fecha de Medicion', 'ID_parcela', 'Biomasa_real Estadistica', 'Validacion']
        df_clf = df_clf.drop(columns=[col for col in features_to_drop_clf if col in df_clf.columns])
        
        target_clf = 'Categoria de Biomasa'
        y_clf = df_clf[target_clf]
        X_clf = df_clf.drop(columns=[target_clf])
        
        return df, X_reg, y_reg, X_clf, y_clf
    except Exception as e:
        print(f"❌ Error cargando datos: {e}")
        return None, None, None, None, None

# Cargar datos
df, X_reg, y_reg, X_clf, y_clf = cargar_y_preparar_datos('Base_Prediccion_Biomasa_Outliers1.xlsx')

if df is not None:
    print(f"\n📊 Resumen de datos:")
    print(f"- Total filas: {df.shape[0]}")
    print(f"- Variables regresión: {X_reg.shape[1]}")
    print(f"- Variables clasificación: {X_clf.shape[1]}")
    print(f"- Distribución clases: {y_clf.value_counts().to_dict()}")

In [ ]:
# Función de preprocesamiento
def preprocesar_datos(X, y, es_clasificacion=False):
    """Preprocesa los datos para modelado"""
    # Codificar variables categóricas
    X_processed = X.copy()
    le_dict = {}
    
    categorical_features = X_processed.select_dtypes(include=['object']).columns
    for col in categorical_features:
        le = LabelEncoder()
        X_processed[col] = le.fit_transform(X_processed[col].astype(str))
        le_dict[col] = le
    
    # Manejar valores nulos
    imputer = SimpleImputer(strategy='median')
    X_processed = pd.DataFrame(imputer.fit_transform(X_processed), columns=X_processed.columns)
    
    # Para clasificación, codificar y
    if es_clasificacion:
        le_target = LabelEncoder()
        y_processed = le_target.fit_transform(y)
        le_dict['target'] = le_target
    else:
        y_processed = y.dropna()
        X_processed = X_processed.loc[y_processed.index]
    
    return X_processed, y_processed, le_dict

# Preprocesar datos
X_reg_proc, y_reg_proc, le_reg = preprocesar_datos(X_reg, y_reg, es_clasificacion=False)
X_clf_proc, y_clf_proc, le_clf = preprocesar_datos(X_clf, y_clf, es_clasificacion=True)

print("✅ Datos preprocesados")
print(f"Regresión: {X_reg_proc.shape}, Clasificación: {X_clf_proc.shape}")

## 2. Modelado Automatizado

In [ ]:
# Función para entrenar modelos de regresión
def entrenar_modelos_regresion(X, y):
    """Entrena múltiples modelos de regresión"""
    # División de datos
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Escalado
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Modelos
    modelos = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
    }
    
    resultados = {}
    
    for nombre, modelo in modelos.items():
        # Entrenar
        if 'Linear' in nombre:
            modelo.fit(X_train_scaled, y_train)
            y_pred = modelo.predict(X_test_scaled)
        else:
            modelo.fit(X_train, y_train)
            y_pred = modelo.predict(X_test)
        
        # Métricas
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        resultados[nombre] = {
            'modelo': modelo,
            'r2': r2,
            'rmse': rmse,
            'y_test': y_test,
            'y_pred': y_pred,
            'scaler': scaler if 'Linear' in nombre else None
        }
        
        print(f"✅ {nombre}: R² = {r2:.4f}, RMSE = {rmse:.4f}")
    
    return resultados, X_train, X_test, y_train, y_test

# Entrenar modelos de regresión
print("🔄 Entrenando modelos de regresión...")
resultados_reg, X_train_reg, X_test_reg, y_train_reg, y_test_reg = entrenar_modelos_regresion(X_reg_proc, y_reg_proc)

In [ ]:
# Función para entrenar modelos de clasificación
def entrenar_modelos_clasificacion(X, y, le_target):
    """Entrena múltiples modelos de clasificación"""
    # División de datos
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    # Escalado
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Modelos
    modelos = {
        'SVM RBF': SVC(kernel='rbf', random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
    }
    
    resultados = {}
    
    for nombre, modelo in modelos.items():
        # Entrenar
        if 'SVM' in nombre:
            modelo.fit(X_train_scaled, y_train)
            y_pred = modelo.predict(X_test_scaled)
        else:
            modelo.fit(X_train, y_train)
            y_pred = modelo.predict(X_test)
        
        # Métricas
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, target_names=le_target.classes_, output_dict=True)
        
        resultados[nombre] = {
            'modelo': modelo,
            'accuracy': accuracy,
            'report': report,
            'y_test': y_test,
            'y_pred': y_pred,
            'scaler': scaler if 'SVM' in nombre else None
        }
        
        print(f"✅ {nombre}: Accuracy = {accuracy:.4f}")
    
    return resultados, X_train, X_test, y_train, y_test

# Entrenar modelos de clasificación
print("\n🔄 Entrenando modelos de clasificación...")
resultados_clf, X_train_clf, X_test_clf, y_train_clf, y_test_clf = entrenar_modelos_clasificacion(
    X_clf_proc, y_clf_proc, le_clf['target']
)

## 3. Generación de Gráficos para Reportes

In [ ]:
# Función para generar gráficos
def generar_graficos_regresion(resultados):
    """Genera gráficos para modelos de regresión"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Comparación de métricas
    modelos = list(resultados.keys())
    r2_scores = [resultados[m]['r2'] for m in modelos]
    rmse_scores = [resultados[m]['rmse'] for m in modelos]
    
    # Gráfico R²
    axes[0,0].bar(modelos, r2_scores, color=['skyblue', 'lightgreen'])
    axes[0,0].set_title('Comparación R² Score', fontsize=14, fontweight='bold')
    axes[0,0].set_ylabel('R² Score')
    axes[0,0].grid(True, alpha=0.3)
    
    # Gráfico RMSE
    axes[0,1].bar(modelos, rmse_scores, color=['salmon', 'gold'])
    axes[0,1].set_title('Comparación RMSE', fontsize=14, fontweight='bold')
    axes[0,1].set_ylabel('RMSE')
    axes[0,1].grid(True, alpha=0.3)
    
    # Predicciones vs Reales (mejor modelo)
    mejor_modelo = max(resultados.keys(), key=lambda k: resultados[k]['r2'])
    y_test = resultados[mejor_modelo]['y_test']
    y_pred = resultados[mejor_modelo]['y_pred']
    
    axes[1,0].scatter(y_test, y_pred, alpha=0.6, color='purple')
    axes[1,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    axes[1,0].set_xlabel('Valores Reales')
    axes[1,0].set_ylabel('Predicciones')
    axes[1,0].set_title(f'Predicciones vs Reales - {mejor_modelo}', fontsize=14, fontweight='bold')
    axes[1,0].grid(True, alpha=0.3)
    
    # Residuos
    residuos = y_test - y_pred
    axes[1,1].scatter(y_pred, residuos, alpha=0.6, color='orange')
    axes[1,1].axhline(y=0, color='r', linestyle='--')
    axes[1,1].set_xlabel('Predicciones')
    axes[1,1].set_ylabel('Residuos')
    axes[1,1].set_title(f'Análisis de Residuos - {mejor_modelo}', fontsize=14, fontweight='bold')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('graficos_regresion.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return mejor_modelo

# Generar gráficos de regresión
print("📊 Generando gráficos de regresión...")
mejor_modelo_reg = generar_graficos_regresion(resultados_reg)

In [ ]:
def generar_graficos_clasificacion(resultados, le_target):
    """Genera gráficos para modelos de clasificación"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Comparación de accuracy
    modelos = list(resultados.keys())
    accuracies = [resultados[m]['accuracy'] for m in modelos]
    
    axes[0,0].bar(modelos, accuracies, color=['lightcoral', 'lightblue'])
    axes[0,0].set_title('Comparación Accuracy', fontsize=14, fontweight='bold')
    axes[0,0].set_ylabel('Accuracy')
    axes[0,0].grid(True, alpha=0.3)
    
    # Mejor modelo para análisis detallado
    mejor_modelo = max(resultados.keys(), key=lambda k: resultados[k]['accuracy'])
    
    # Matriz de confusión
    from sklearn.metrics import confusion_matrix
    y_test = resultados[mejor_modelo]['y_test']
    y_pred = resultados[mejor_modelo]['y_pred']
    cm = confusion_matrix(y_test, y_pred)
    
    im = axes[0,1].imshow(cm, interpolation='nearest', cmap='Blues')
    axes[0,1].set_title(f'Matriz Confusión - {mejor_modelo}', fontsize=14, fontweight='bold')
    
    # Añadir texto a la matriz
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            axes[0,1].text(j, i, format(cm[i, j], 'd'),
                         horizontalalignment="center",
                         color="white" if cm[i, j] > thresh else "black")
    
    axes[0,1].set_ylabel('Clase Real')
    axes[0,1].set_xlabel('Clase Predicha')
    axes[0,1].set_xticks(range(len(le_target.classes_)))
    axes[0,1].set_yticks(range(len(le_target.classes_)))
    axes[0,1].set_xticklabels(le_target.classes_)
    axes[0,1].set_yticklabels(le_target.classes_)
    
    # Distribución de clases
    unique, counts = np.unique(y_test, return_counts=True)
    class_names = [le_target.classes_[i] for i in unique]
    
    axes[1,0].pie(counts, labels=class_names, autopct='%1.1f%%', startangle=90)
    axes[1,0].set_title('Distribución de Clases en Test', fontsize=14, fontweight='bold')
    
    # Métricas por clase
    report = resultados[mejor_modelo]['report']
    clases = le_target.classes_
    precision = [report[clase]['precision'] for clase in clases]
    recall = [report[clase]['recall'] for clase in clases]
    f1 = [report[clase]['f1-score'] for clase in clases]
    
    x = np.arange(len(clases))
    width = 0.25
    
    axes[1,1].bar(x - width, precision, width, label='Precision', alpha=0.8)
    axes[1,1].bar(x, recall, width, label='Recall', alpha=0.8)
    axes[1,1].bar(x + width, f1, width, label='F1-Score', alpha=0.8)
    
    axes[1,1].set_xlabel('Clases')
    axes[1,1].set_ylabel('Score')
    axes[1,1].set_title('Métricas por Clase', fontsize=14, fontweight='bold')
    axes[1,1].set_xticks(x)
    axes[1,1].set_xticklabels(clases)
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('graficos_clasificacion.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return mejor_modelo

# Generar gráficos de clasificación
print("📊 Generando gráficos de clasificación...")
mejor_modelo_clf = generar_graficos_clasificacion(resultados_clf, le_clf['target'])

## 4. Generación Automatizada de Reportes en Excel

In [ ]:
def crear_reporte_excel_automatizado(df, resultados_reg, resultados_clf, le_clf):
    """Crea un reporte completo en Excel con múltiples hojas"""
    
    # Crear archivo Excel
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_archivo = f'Reporte_Biomasa_ML_{timestamp}.xlsx'
    
    # Usar ExcelWriter de pandas para múltiples hojas
    with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
        
        # HOJA 1: Resumen Ejecutivo
        resumen_data = {
            'Métrica': [
                'Total de Registros',
                'Variables Predictoras',
                'Mejor Modelo Regresión',
                'R² Mejor Modelo',
                'RMSE Mejor Modelo',
                'Mejor Modelo Clasificación',
                'Accuracy Mejor Modelo',
                'Fecha Generación Reporte'
            ],
            'Valor': [
                df.shape[0],
                len([col for col in df.columns if col not in ['Fecha de Medicion', 'ID_parcela', 'Categoria de Biomasa', 'Biomasa_real Estadistica', 'Validacion']]),
                max(resultados_reg.keys(), key=lambda k: resultados_reg[k]['r2']),
                f"{max([resultados_reg[k]['r2'] for k in resultados_reg]):.4f}",
                f"{min([resultados_reg[k]['rmse'] for k in resultados_reg]):.4f}",
                max(resultados_clf.keys(), key=lambda k: resultados_clf[k]['accuracy']),
                f"{max([resultados_clf[k]['accuracy'] for k in resultados_clf]):.4f}",
                datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            ]
        }
        
        resumen_df = pd.DataFrame(resumen_data)
        resumen_df.to_excel(writer, sheet_name='Resumen_Ejecutivo', index=False)
        
        # HOJA 2: Resultados Regresión
        reg_results = pd.DataFrame({
            'Modelo': list(resultados_reg.keys()),
            'R²_Score': [resultados_reg[k]['r2'] for k in resultados_reg],
            'RMSE': [resultados_reg[k]['rmse'] for k in resultados_reg]
        }).sort_values('R²_Score', ascending=False)
        
        reg_results.to_excel(writer, sheet_name='Resultados_Regresion', index=False)
        
        # HOJA 3: Resultados Clasificación
        clf_results = pd.DataFrame({
            'Modelo': list(resultados_clf.keys()),
            'Accuracy': [resultados_clf[k]['accuracy'] for k in resultados_clf]
        }).sort_values('Accuracy', ascending=False)
        
        clf_results.to_excel(writer, sheet_name='Resultados_Clasificacion', index=False)
        
        # HOJA 4: Análisis de Datos
        analisis_data = {
            'Variable': [],
            'Tipo': [],
            'Valores_Nulos': [],
            'Media': [],
            'Desviacion_Std': []
        }
        
        for col in df.columns:
            if col not in ['Fecha de Medicion', 'ID_parcela', 'Validacion']:
                analisis_data['Variable'].append(col)
                analisis_data['Tipo'].append(str(df[col].dtype))
                analisis_data['Valores_Nulos'].append(df[col].isnull().sum())
                
                if df[col].dtype in ['int64', 'float64']:
                    analisis_data['Media'].append(f"{df[col].mean():.4f}")
                    analisis_data['Desviacion_Std'].append(f"{df[col].std():.4f}")
                else:
                    analisis_data['Media'].append('N/A')
                    analisis_data['Desviacion_Std'].append('N/A')
        
        analisis_df = pd.DataFrame(analisis_data)
        analisis_df.to_excel(writer, sheet_name='Analisis_Datos', index=False)
        
        # HOJA 5: Datos Originales (muestra)
        df.head(100).to_excel(writer, sheet_name='Datos_Muestra', index=False)
        
        # HOJA 6: Predicciones del Mejor Modelo
        mejor_reg = max(resultados_reg.keys(), key=lambda k: resultados_reg[k]['r2'])
        mejor_clf = max(resultados_clf.keys(), key=lambda k: resultados_clf[k]['accuracy'])
        
        predicciones_data = {
            'Indice': range(len(resultados_reg[mejor_reg]['y_test'])),
            'Biomasa_Real': resultados_reg[mejor_reg]['y_test'].values,
            'Biomasa_Predicha': resultados_reg[mejor_reg]['y_pred'],
            'Error_Absoluto': np.abs(resultados_reg[mejor_reg]['y_test'].values - resultados_reg[mejor_reg]['y_pred'])
        }
        
        predicciones_df = pd.DataFrame(predicciones_data)
        predicciones_df.to_excel(writer, sheet_name='Predicciones', index=False)
    
    print(f"✅ Reporte Excel creado: {nombre_archivo}")
    return nombre_archivo

# Crear reporte Excel
print("📄 Generando reporte Excel automatizado...")
archivo_excel = crear_reporte_excel_automatizado(df, resultados_reg, resultados_clf, le_clf)

In [ ]:
# Función para dar formato avanzado al Excel usando openpyxl
def formatear_excel_avanzado(nombre_archivo):
    """Aplica formato avanzado al archivo Excel"""
    
    # Cargar el libro
    wb = openpyxl.load_workbook(nombre_archivo)
    
    # Definir estilos
    header_font = Font(bold=True, color="FFFFFF")
    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    center_alignment = Alignment(horizontal="center", vertical="center")
    border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    # Formatear cada hoja
    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        
        # Formatear headers
        for cell in ws[1]:
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = center_alignment
            cell.border = border
        
        # Ajustar ancho de columnas
        for column in ws.columns:
            max_length = 0
            column_letter = column[0].column_letter
            
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            
            adjusted_width = min(max_length + 2, 50)
            ws.column_dimensions[column_letter].width = adjusted_width
        
        # Aplicar bordes a todas las celdas con datos
        max_row = ws.max_row
        max_col = ws.max_column
        
        for row in range(1, max_row + 1):
            for col in range(1, max_col + 1):
                ws.cell(row=row, column=col).border = border
                
                # Centrar contenido numérico
                if row > 1:  # No headers
                    cell_value = ws.cell(row=row, column=col).value
                    if isinstance(cell_value, (int, float)):
                        ws.cell(row=row, column=col).alignment = center_alignment
    
    # Agregar gráficos si es posible
    try:
        # Gráfico en hoja de regresión
        ws_reg = wb['Resultados_Regresion']
        
        # Crear gráfico de barras para R²
        chart = BarChart()
        chart.title = "Comparación R² Score por Modelo"
        chart.y_axis.title = "R² Score"
        chart.x_axis.title = "Modelos"
        
        # Datos para el gráfico
        data = Reference(ws_reg, min_col=2, min_row=1, max_row=ws_reg.max_row, max_col=2)
        cats = Reference(ws_reg, min_col=1, min_row=2, max_row=ws_reg.max_row)
        
        chart.add_data(data, titles_from_data=True)
        chart.set_categories(cats)
        
        ws_reg.add_chart(chart, "E2")
        
    except Exception as e:
        print(f"⚠️ No se pudieron agregar gráficos: {e}")
    
    # Guardar cambios
    wb.save(nombre_archivo)
    print(f"✅ Formato avanzado aplicado a {nombre_archivo}")

# Aplicar formato avanzado
print("🎨 Aplicando formato avanzado...")
formatear_excel_avanzado(archivo_excel)

## 5. Dashboard Simple con Resultados

In [ ]:
def crear_dashboard_biomasa():
    """Crea un dashboard simple con los resultados principales"""
    
    # Configurar el dashboard
    fig = plt.figure(figsize=(20, 12))
    fig.suptitle('🌱 DASHBOARD PREDICTIVO DE BIOMASA - MACHINE LEARNING', 
                 fontsize=24, fontweight='bold', y=0.98)
    
    # Layout del dashboard
    gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)
    
    # 1. KPIs principales (fila superior)
    ax_kpi1 = fig.add_subplot(gs[0, 0])
    ax_kpi2 = fig.add_subplot(gs[0, 1])
    ax_kpi3 = fig.add_subplot(gs[0, 2])
    ax_kpi4 = fig.add_subplot(gs[0, 3])
    
    # KPI 1: Total de registros
    ax_kpi1.text(0.5, 0.5, f'{df.shape[0]}', ha='center', va='center', 
                fontsize=36, fontweight='bold', color='#2E86AB')
    ax_kpi1.text(0.5, 0.2, 'Total Registros', ha='center', va='center', 
                fontsize=12, fontweight='bold')
    ax_kpi1.set_xlim(0, 1)
    ax_kpi1.set_ylim(0, 1)
    ax_kpi1.axis('off')
    ax_kpi1.add_patch(plt.Rectangle((0.05, 0.05), 0.9, 0.9, fill=False, 
                                   edgecolor='#2E86AB', linewidth=3))
    
    # KPI 2: Mejor R²
    mejor_r2 = max([resultados_reg[k]['r2'] for k in resultados_reg])
    ax_kpi2.text(0.5, 0.5, f'{mejor_r2:.3f}', ha='center', va='center', 
                fontsize=36, fontweight='bold', color='#A23B72')
    ax_kpi2.text(0.5, 0.2, 'Mejor R² Score', ha='center', va='center', 
                fontsize=12, fontweight='bold')
    ax_kpi2.set_xlim(0, 1)
    ax_kpi2.set_ylim(0, 1)
    ax_kpi2.axis('off')
    ax_kpi2.add_patch(plt.Rectangle((0.05, 0.05), 0.9, 0.9, fill=False, 
                                   edgecolor='#A23B72', linewidth=3))
    
    # KPI 3: Mejor Accuracy
    mejor_acc = max([resultados_clf[k]['accuracy'] for k in resultados_clf])
    ax_kpi3.text(0.5, 0.5, f'{mejor_acc:.3f}', ha='center', va='center', 
                fontsize=36, fontweight='bold', color='#F18F01')
    ax_kpi3.text(0.5, 0.2, 'Mejor Accuracy', ha='center', va='center', 
                fontsize=12, fontweight='bold')
    ax_kpi3.set_xlim(0, 1)
    ax_kpi3.set_ylim(0, 1)
    ax_kpi3.axis('off')
    ax_kpi3.add_patch(plt.Rectangle((0.05, 0.05), 0.9, 0.9, fill=False, 
                                   edgecolor='#F18F01', linewidth=3))
    
    # KPI 4: Variables predictoras
    n_vars = len([col for col in df.columns if col not in ['Fecha de Medicion', 'ID_parcela', 'Categoria de Biomasa', 'Biomasa_real Estadistica', 'Validacion']])
    ax_kpi4.text(0.5, 0.5, f'{n_vars}', ha='center', va='center', 
                fontsize=36, fontweight='bold', color='#C73E1D')
    ax_kpi4.text(0.5, 0.2, 'Variables Predictoras', ha='center', va='center', 
                fontsize=12, fontweight='bold')
    ax_kpi4.set_xlim(0, 1)
    ax_kpi4.set_ylim(0, 1)
    ax_kpi4.axis('off')
    ax_kpi4.add_patch(plt.Rectangle((0.05, 0.05), 0.9, 0.9, fill=False, 
                                   edgecolor='#C73E1D', linewidth=3))
    
    # 2. Gráficos de rendimiento (fila media)
    ax_reg = fig.add_subplot(gs[1, :2])
    ax_clf = fig.add_subplot(gs[1, 2:])
    
    # Rendimiento modelos regresión
    modelos_reg = list(resultados_reg.keys())
    r2_scores = [resultados_reg[m]['r2'] for m in modelos_reg]
    colors_reg = ['#2E86AB', '#A23B72']
    
    bars_reg = ax_reg.bar(modelos_reg, r2_scores, color=colors_reg)
    ax_reg.set_title('📊 Rendimiento Modelos de Regresión', fontsize=16, fontweight='bold')
    ax_reg.set_ylabel('R² Score', fontsize=12)
    ax_reg.grid(True, alpha=0.3)
    
    # Añadir valores en las barras
    for bar, score in zip(bars_reg, r2_scores):
        ax_reg.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                   f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # Rendimiento modelos clasificación
    modelos_clf = list(resultados_clf.keys())
    acc_scores = [resultados_clf[m]['accuracy'] for m in modelos_clf]
    colors_clf = ['#F18F01', '#C73E1D']
    
    bars_clf = ax_clf.bar(modelos_clf, acc_scores, color=colors_clf)
    ax_clf.set_title('🎯 Rendimiento Modelos de Clasificación', fontsize=16, fontweight='bold')
    ax_clf.set_ylabel('Accuracy', fontsize=12)
    ax_clf.grid(True, alpha=0.3)
    
    # Añadir valores en las barras
    for bar, score in zip(bars_clf, acc_scores):
        ax_clf.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                   f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 3. Análisis de datos (fila inferior)
    ax_dist = fig.add_subplot(gs[2, :2])
    ax_corr = fig.add_subplot(gs[2, 2:])
    
    # Distribución de biomasa
    biomasa_vals = df['Biomasa_real Estadistica'].dropna()
    ax_dist.hist(biomasa_vals, bins=20, color='#2E86AB', alpha=0.7, edgecolor='black')
    ax_dist.set_title('📈 Distribución de Biomasa Real', fontsize=16, fontweight='bold')
    ax_dist.set_xlabel('Biomasa (kg/ha)', fontsize=12)
    ax_dist.set_ylabel('Frecuencia', fontsize=12)
    ax_dist.grid(True, alpha=0.3)
    
    # Correlación entre variables principales
    vars_numericas = ['NDVI Outlier Manual', 'NDRE Outlier Manual', 
                     'PRECIPITACION Outlier Manual', 'Biomasa_real Estadistica']
    vars_disponibles = [v for v in vars_numericas if v in df.columns]
    
    if len(vars_disponibles) > 1:
        corr_matrix = df[vars_disponibles].corr()
        im = ax_corr.imshow(corr_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
        
        # Añadir texto de correlación
        for i in range(len(vars_disponibles)):
            for j in range(len(vars_disponibles)):
                text = ax_corr.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                                   ha="center", va="center", color="black", fontweight='bold')
        
        ax_corr.set_xticks(range(len(vars_disponibles)))
        ax_corr.set_yticks(range(len(vars_disponibles)))
        ax_corr.set_xticklabels([v.replace(' Outlier Manual', '') for v in vars_disponibles], rotation=45)
        ax_corr.set_yticklabels([v.replace(' Outlier Manual', '') for v in vars_disponibles])
        ax_corr.set_title('🔗 Matriz de Correlación', fontsize=16, fontweight='bold')
        
        # Colorbar
        cbar = plt.colorbar(im, ax=ax_corr, shrink=0.6)
        cbar.set_label('Correlación', fontsize=10)
    
    # Añadir timestamp
    fig.text(0.99, 0.01, f'Generado: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}', 
             ha='right', va='bottom', fontsize=10, style='italic')
    
    plt.savefig('dashboard_biomasa.png', dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()
    
    print("✅ Dashboard generado y guardado como 'dashboard_biomasa.png'")

# Crear dashboard
print("📊 Creando dashboard...")
crear_dashboard_biomasa()

## 6. Función de Predicción para Nuevos Datos

In [ ]:
def crear_sistema_prediccion_excel():
    """Crea un archivo Excel para realizar predicciones con nuevos datos"""
    
    # Crear archivo para predicciones
    wb_pred = Workbook()
    
    # Hoja de instrucciones
    ws_instrucciones = wb_pred.active
    ws_instrucciones.title = "Instrucciones"
    
    instrucciones = [
        ["SISTEMA DE PREDICCIÓN DE BIOMASA"],
        ["="*50],
        [""],
        ["INSTRUCCIONES DE USO:"],
        ["1. Vaya a la hoja 'Nuevos_Datos'"],
        ["2. Complete los valores en las columnas correspondientes"],
        ["3. Los resultados aparecerán automáticamente en las columnas de predicción"],
        [""],
        ["VARIABLES REQUERIDAS:"],
        ["- NDVI Outlier Manual: Valor entre 0 y 1"],
        ["- NDRE Outlier Manual: Valor entre 0 y 1"],
        ["- PRECIPITACION Outlier Manual: Milímetros de lluvia"],
        ["- DIAS SIN LLUVIA Estadistica: Número de días"],
        ["- Tipo_suelo: 0=Arcilloso, 1=Arenoso, 2=Franco"],
        [""],
        ["RESULTADOS:"],
        ["- Biomasa_Predicha: Valor estimado en kg/ha"],
        ["- Categoria_Predicha: Baja/Media/Alta"],
        ["- Confianza: Nivel de confianza del modelo"]
    ]
    
    for row_num, row_data in enumerate(instrucciones, 1):
        ws_instrucciones.cell(row=row_num, column=1, value=row_data[0])
        if row_num == 1:
            ws_instrucciones.cell(row=row_num, column=1).font = Font(bold=True, size=16)
        elif "INSTRUCCIONES" in row_data[0] or "VARIABLES" in row_data[0] or "RESULTADOS" in row_data[0]:
            ws_instrucciones.cell(row=row_num, column=1).font = Font(bold=True, size=12)
    
    # Hoja para nuevos datos
    ws_datos = wb_pred.create_sheet("Nuevos_Datos")
    
    # Headers
    headers = [
        "ID", "NDVI Outlier Manual", "NDRE Outlier Manual", 
        "PRECIPITACION Outlier Manual", "DIAS SIN LLUVIA Estadistica", 
        "Tipo_suelo", "Biomasa_Predicha", "Categoria_Predicha", "Confianza_Modelo"
    ]
    
    for col_num, header in enumerate(headers, 1):
        cell = ws_datos.cell(row=1, column=col_num, value=header)
        cell.font = Font(bold=True, color="FFFFFF")
        cell.fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
        cell.alignment = Alignment(horizontal="center")
    
    # Datos de ejemplo
    ejemplos = [
        [1, 0.65, 0.58, 45.2, 5, 1, "=PREDICCIÓN", "=CLASIFICACIÓN", "=CONFIANZA"],
        [2, 0.72, 0.61, 38.7, 3, 0, "=PREDICCIÓN", "=CLASIFICACIÓN", "=CONFIANZA"],
        [3, 0.55, 0.49, 52.1, 8, 2, "=PREDICCIÓN", "=CLASIFICACIÓN", "=CONFIANZA"]
    ]
    
    for row_num, row_data in enumerate(ejemplos, 2):
        for col_num, value in enumerate(row_data, 1):
            ws_datos.cell(row=row_num, column=col_num, value=value)
    
    # Ajustar ancho de columnas
    for column in ws_datos.columns:
        max_length = max(len(str(cell.value)) for cell in column)
        ws_datos.column_dimensions[column[0].column_letter].width = max_length + 2
    
    # Hoja de modelos (con métricas)
    ws_modelos = wb_pred.create_sheet("Info_Modelos")
    
    # Información de modelos
    mejor_reg = max(resultados_reg.keys(), key=lambda k: resultados_reg[k]['r2'])
    mejor_clf = max(resultados_clf.keys(), key=lambda k: resultados_clf[k]['accuracy'])
    
    info_modelos = [
        ["INFORMACIÓN DE MODELOS ENTRENADOS"],
        [""],
        ["MODELO DE REGRESIÓN (Predicción de Biomasa):"],
        [f"Algoritmo: {mejor_reg}"],
        [f"R² Score: {resultados_reg[mejor_reg]['r2']:.4f}"],
        [f"RMSE: {resultados_reg[mejor_reg]['rmse']:.4f}"],
        [""],
        ["MODELO DE CLASIFICACIÓN (Categoría de Biomasa):"],
        [f"Algoritmo: {mejor_clf}"],
        [f"Accuracy: {resultados_clf[mejor_clf]['accuracy']:.4f}"],
        [""],
        ["FECHA DE ENTRENAMIENTO:"],
        [datetime.now().strftime("%Y-%m-%d %H:%M:%S")],
        [""],
        ["TOTAL DE DATOS DE ENTRENAMIENTO:"],
        [f"{len(y_train_reg)} muestras"]
    ]
    
    for row_num, row_data in enumerate(info_modelos, 1):
        ws_modelos.cell(row=row_num, column=1, value=row_data[0])
        if "INFORMACIÓN" in row_data[0] or "MODELO DE" in row_data[0]:
            ws_modelos.cell(row=row_num, column=1).font = Font(bold=True)
    
    # Guardar archivo
    nombre_pred = f'Sistema_Prediccion_Biomasa_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx'
    wb_pred.save(nombre_pred)
    
    print(f"✅ Sistema de predicción creado: {nombre_pred}")
    return nombre_pred

# Crear sistema de predicción
print("🔮 Creando sistema de predicción...")
archivo_prediccion = crear_sistema_prediccion_excel()

## 7. Resumen y Archivos Generados

In [ ]:
def mostrar_resumen_final():
    """Muestra un resumen completo del proceso y archivos generados"""
    
    print("\n" + "="*80)
    print("🎉 PROCESO COMPLETADO - RESUMEN FINAL")
    print("="*80)
    
    print("\n📊 DATOS PROCESADOS:")
    print(f"   • Total de registros: {df.shape[0]}")
    print(f"   • Variables predictoras: {len([col for col in df.columns if col not in ['Fecha de Medicion', 'ID_parcela', 'Categoria de Biomasa', 'Biomasa_real Estadistica', 'Validacion']])}")
    print(f"   • División entrenamiento/prueba: 70/30")
    
    print("\n🤖 MODELOS ENTRENADOS:")
    print("   REGRESIÓN:")
    for modelo, resultado in resultados_reg.items():
        print(f"     • {modelo}: R² = {resultado['r2']:.4f}, RMSE = {resultado['rmse']:.4f}")
    
    print("   CLASIFICACIÓN:")
    for modelo, resultado in resultados_clf.items():
        print(f"     • {modelo}: Accuracy = {resultado['accuracy']:.4f}")
    
    print("\n🏆 MEJORES MODELOS:")
    mejor_reg = max(resultados_reg.keys(), key=lambda k: resultados_reg[k]['r2'])
    mejor_clf = max(resultados_clf.keys(), key=lambda k: resultados_clf[k]['accuracy'])
    print(f"   • Regresión: {mejor_reg} (R² = {resultados_reg[mejor_reg]['r2']:.4f})")
    print(f"   • Clasificación: {mejor_clf} (Accuracy = {resultados_clf[mejor_clf]['accuracy']:.4f})")
    
    print("\n📁 ARCHIVOS GENERADOS:")
    print(f"   📄 {archivo_excel}")
    print("      → Reporte completo con múltiples hojas")
    print("      → Métricas de modelos y análisis de datos")
    print("      → Formato profesional con gráficos")
    print()
    print(f"   🔮 {archivo_prediccion}")
    print("      → Sistema de predicción para nuevos datos")
    print("      → Plantilla para uso operativo")
    print("      → Instrucciones incluidas")
    print()
    print("   📊 dashboard_biomasa.png")
    print("      → Dashboard visual con KPIs principales")
    print("      → Gráficos de rendimiento de modelos")
    print("      → Análisis exploratorio de datos")
    print()
    print("   📈 graficos_regresion.png")
    print("      → Análisis detallado de modelos de regresión")
    print()
    print("   🎯 graficos_clasificacion.png")
    print("      → Análisis detallado de modelos de clasificación")
    
    print("\n⚡ CARACTERÍSTICAS IMPLEMENTADAS:")
    print("   ✅ Carga automatizada de datos desde Excel")
    print("   ✅ Preprocesamiento completo (nulos, codificación, escalado)")
    print("   ✅ Entrenamiento de múltiples modelos ML")
    print("   ✅ Generación automática de reportes Excel")
    print("   ✅ Formato profesional con openpyxl")
    print("   ✅ Dashboard visual con matplotlib/seaborn")
    print("   ✅ Sistema de predicción para nuevos datos")
    print("   ✅ Métricas de evaluación completas")
    print("   ✅ Visualizaciones de alta calidad")
    
    print("\n🔧 TECNOLOGÍAS UTILIZADAS:")
    print("   • pandas & openpyxl: Automatización de reportes Excel")
    print("   • scikit-learn: Modelos de Machine Learning")
    print("   • matplotlib & seaborn: Visualizaciones")
    print("   • Excel: Formato profesional y operativo")
    
    print("\n" + "="*80)
    print("🚀 ¡SISTEMA DE PREDICCIÓN DE BIOMASA LISTO PARA USAR!")
    print("="*80)

# Mostrar resumen final
mostrar_resumen_final()

## 8. Ejemplo de Uso del Sistema

Para usar el sistema de predicción:

1. **Abrir** el archivo `Sistema_Prediccion_Biomasa_[timestamp].xlsx`
2. **Ir** a la hoja "Nuevos_Datos"
3. **Completar** los valores de las variables predictoras
4. **Ver** las predicciones automáticas en las columnas correspondientes

### Variables de entrada:
- **NDVI Outlier Manual**: Índice de vegetación (0-1)
- **NDRE Outlier Manual**: Índice de borde rojo (0-1) 
- **PRECIPITACION Outlier Manual**: Precipitación en mm
- **DIAS SIN LLUVIA Estadistica**: Días consecutivos sin lluvia
- **Tipo_suelo**: 0=Arcilloso, 1=Arenoso, 2=Franco

### Resultados:
- **Biomasa_Predicha**: Valor estimado en kg/ha
- **Categoria_Predicha**: Baja/Media/Alta
- **Confianza_Modelo**: Nivel de confianza

El sistema está completamente automatizado y listo para uso operativo.